# Classify operating expenditure types

Ideally I would have done this more programmatically, but in the interest of time these were coded manually

In [1]:
import numpy as np
import pandas as pd
from pandasql import sqldf

In [2]:
# Data set defined in explore_and_clean_filing_data.ipynb
df = pd.read_csv('../data/processed/expenditure_purpose_coding.csv')

/var/folders/cl/bdrh3m8j0fdfj6zm7zhhc2ch0000gn/T/ipykernel_16060/3502164859.py:1: DtypeWarning: Columns (12,17,22,24,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/processed/expenditure_purpose_coding.csv')


In [3]:
# clean strings
def clean_strings(text):
    if pd.isnull(text):
        return ""
    text = text.lower() 
    text = re.sub(r"\d+",'',text)
    text = text.replace(f"{string.punctuation}",'')
    text = text.strip()
    return text 

In [ ]:
# normalize strings
df['clean_vendor']=df['payee_organization_name'].apply(clean_strings)
df['clean_purpose']=df['expenditure_purpose_descrip'].apply(clean_strings)

In [6]:
df[['payee_organization_name','clean_vendor']].sample(25)

,payee_organization_name,clean_vendor
536140,GODADDY COM,godaddy com
150965,USPS,usps
569107,NORTH COUNTRY STRATEGIES LLC,north country strategies llc
544279,CFS COMPLIANCE,cfs compliance
98381,CAPITOL ONE N A,capitol one n a
396194,NaN,
359415,COMMITTEE TO ELECT JAKE JOHNSON,committee to elect jake johnson
10007,STRATHDEE GROUP,strathdee group
467682,NaN,
302184,TEXAS HOUSE REPUBLICAN CAUCUS PAC,texas house republican caucus pac


In [7]:
df[['expenditure_purpose_descrip','clean_purpose']].sample(25)

,expenditure_purpose_descrip,clean_purpose
191739,STIPEND,stipend
403748,COMPLIANCE SERVICES,compliance services
182266,PROCESSING FEE,processing fee
218559,FUNDRAISING EXPENSES,fundraising expenses
340506,COMPLIANCE CONSULTING,compliance consulting
110343,CREDIT CARD MERCHANT FEE,credit card merchant fee
317735,PRINTING,printing
66961,CONSULTANT LIVE TRAINING SERVICES,consultant live training services
164681,SOFTWARE RENTAL,software rental
342723,SALARY,salary


In [ ]:
# Using clean vendor and clean purpose - categorize operating expenditures to larger categories
conditions = [
    df['clean_vendor'].str.contains('coinbase', na=False)  
    , df['clean_vendor'].str.contains('actblue', na=False)    
    , df['clean_vendor'].str.contains('act blue', na=False)   
    , df['clean_vendor'].str.contains('winred', na=False)   
    , df['clean_vendor'].str.contains('win red', na=False)   
    , df['clean_vendor'].str.contains('anedot', na=False)   
    , df['clean_purpose'].str.contains('direct mail', na=False)   
    , df['clean_vendor'].str.contains('airline', na=False)   
    , df['clean_vendor'].str.contains('amtrak', na=False)   
    , df['clean_purpose'].str.contains('advertisements', na=False)   
    , df['clean_purpose'].str.contains('earmark', na=False)   
    , df['clean_purpose'].str.contains('advertisements', na=False)   
    , df['clean_purpose'].str.contains('salary', na=False)   
    , df['clean_purpose'].str.contains('salaries', na=False)   
    , df['clean_purpose'].str.contains('payroll', na=False)   
    , df['clean_purpose'].str.contains('stipend', na=False) 
    , df['clean_vendor'].str.contains('lyft', na=False)
    , df['clean_purpose'].str.contains('travel', na=False)   
    , df['clean_purpose'].str.contains('nonfederal contribution', na=False) 
    , df['clean_purpose'].str.contains('fundrais', na=False) 
    , df['clean_purpose'].str.contains('processing fee',na=False)
    , df['clean_purpose'].str.contains('transportation', na=False)  
    , df['clean_vendor'].str.contains('bank', na=False)
    , df['clean_vendor'].str.contains('american express', na=False)  
    , df['clean_purpose'].str.contains('advert', na=False) 
    , df['clean_purpose'].str.contains('compliance',na=False)  
    , df['clean_purpose'].str.contains('consulting', na=False)   
    , df['clean_purpose'].str.contains('services', na=False) 
    , df['clean_purpose'].str.contains('credit card fee',na=False)  
    ,df['clean_purpose'].str.contains(' list ',na=False) 
    ,df['clean_vendor'].str.contains('ngp van',na=False) 
    ,df['clean_purpose'].str.contains('supplies',na=False) 
    ,df['clean_purpose'].str.contains('postage',na=False) 
    ,df['clean_vendor'].str.contains('stripe',na=False) 
    ,df['clean_vendor'].str.contains('shopify',na=False) 
    ,df['clean_purpose'].str.contains('contribution',na=False) 
    ,df['clean_vendor'].str.contains('friends of',na=False)
    ,df['clean_purpose'].str.contains('refund',na=False) 
    ,df['clean_purpose'].str.contains('mileage',na=False) 
    ,df['clean_purpose'].str.contains('software',na=False)  
    ,df['clean_purpose'].str.contains('list acquisition',na=False)  
    ,df['clean_purpose'].str.contains('canvassing',na=False)  
    ,df['clean_purpose'].str.contains('gotv',na=False) 
    ,df['clean_purpose'].str.contains('subscription',na=False)  
    ,df['clean_purpose'].str.contains('reimbursement',na=False) 
    ,df['clean_purpose'].str.contains('bank fee',na=False)
    ,df['clean_purpose'].str.contains('meals',na=False)  
    ,df['clean_vendor'].str.contains('uber',na=False)  
    ,df['clean_purpose'].str.contains('leads phone lists',na=False) 
    ,df['clean_purpose'].str.contains('credit card transaction fees',na=False) 
    ,df['clean_vendor'].str.contains('print',na=False) 
    ,df['clean_purpose'].str.contains('parking',na=False)
    ,df['clean_purpose'].str.contains('shipping',na=False)
    ,df['clean_vendor'].str.contains('nationbuild',na=False)
    ,df['clean_vendor'].str.contains('meta platforms inc',na=False)
    ,df['clean_purpose'].str.contains('telemarketing',na=False)
    ,df['clean_vendor'].str.contains('democracy engine',na=False)
    ,df['clean_purpose'].str.contains('print',na=False) 
    ,df['clean_vendor'].str.contains('paypal',na=False) 
    ,df['clean_purpose'].str.contains('list rental',na=False) 
    ,df['clean_vendor'].str.contains('compliance',na=False) 
    ,df['clean_vendor'].str.contains('square',na=False) 
    ,df['clean_vendor'].str.contains('getthru',na=False)  
    ,df['clean_vendor'].str.contains('get thru',na=False)  
    ,df['clean_vendor'].str.contains('bumperactive',na=False)
    ,df['clean_vendor'].str.contains('tatango',na=False)
    ,df['clean_vendor'].str.contains('cmdi',na=False)
    ,df['clean_vendor'].str.contains('revv',na=False)
    ,df['clean_vendor'].str.contains('club for growth',na=False) 
    ,df['clean_vendor'].str.contains('league of conservation voters inc',na=False)
    ,df['clean_vendor'].str.contains('egencia',na=False)
    ,df['clean_vendor'].str.contains('wepay inc',na=False)
    ,df['clean_vendor'].str.contains('rallypay',na=False)
    ,df['clean_purpose'].str.contains('insurance',na=False)
    ,df['clean_purpose'].str.contains('direct marketing',na=False)
    ,df['clean_vendor'].str.contains(' for ',na=False) 
    ,df['clean_purpose'].str.contains('wage',na=False)
    ,df['clean_purpose'].str.contains('staff',na=False)
    ,df['clean_purpose'].str.contains('employee',na=False)
    ,df['clean_purpose'].str.contains('healthcare',na=False)
    ,df['clean_purpose'].str.contains('per diem',na=False)
    ,df['clean_purpose'].str.contains('database',na=False)
    ,df['clean_purpose'].str.contains('tech',na=False)
    ,df['clean_vendor'].str.contains('google',na=False) 
    ,df['clean_vendor'].str.contains('wells fargo',na=False) 
    ,df['clean_purpose'].str.contains('personnel',na=False)
    ,df['clean_purpose'].str.contains('consultant',na=False)
    ,df['clean_purpose'].str.contains('food',na=False)
    ,df['clean_purpose'].str.contains('legal',na=False)
    ,df['clean_purpose'].str.contains('donation processing',na=False)
    ,df['clean_vendor'].str.contains('fundraising',na=False)
    ,df['clean_purpose'].str.contains('digital marketingg',na=False)
    ,df['clean_purpose'].str.contains('stamps',na=False)
    ,df['clean_purpose'].str.contains('catering',na=False)
    ,df['clean_purpose'].str.contains('lodging',na=False)
    ,df['clean_vendor'].str.contains('everyaction',na=False) 
    ,df['clean_vendor'].str.contains('exxon',na=False) 
    ,df['clean_purpose'].str.contains('donation',na=False)
    ,df['clean_purpose'].str.contains('hotel',na=False)
    ,df['clean_vendor'].str.contains('campaign',na=False) 
    ,df['clean_vendor'].str.contains('treasury',na=False) 
    ,df['clean_purpose'].str.contains('tax',na=False)
    ,df['clean_vendor'].str.contains('capital one',na=False)
    ,df['clean_purpose'].str.contains('digital field',na=False) 
    ,df['clean_purpose'].str.contains('phone bank',na=False) 
    ,df['clean_purpose'].str.contains('phonebank',na=False)
    ,df['clean_purpose'].str.contains('research',na=False) 
    ,df['clean_purpose'].str.contains('telephone',na=False)
    ,df['clean_purpose'].str.contains('media production',na=False) 
    ,df['clean_vendor'].str.contains('ace specialties',na=False)
    ,df['clean_purpose'].str.contains('marketing',na=False)
    ,df['clean_purpose'].str.contains('caging',na=False)
    ,df['clean_vendor'].str.contains('paragon',na=False)
    ,df['clean_vendor'].str.contains('united states postal service', na=False) 
    ,df['clean_purpose'].str.contains('ad production',na=False)
    ,df['clean_purpose'].str.contains('bookkeeping',na=False)
    ,df['clean_purpose'].str.contains('utilities',na=False)
    ,df['clean_purpose'].str.contains('rent',na=False)
    ,df['clean_purpose'].str.contains('office',na=False)
    ,df['clean_purpose'].str.contains('web',na=False)
    ,df['clean_vendor'].str.contains('verizon', na=False) 
    ,df['clean_vendor'].str.contains('at t', na=False)
    ,df['clean_purpose'].str.contains('credit card processing',na=False) 
    ,df['clean_vendor'].str.contains('doordash', na=False)
    ,df['clean_vendor'].str.contains('adp', na=False)
    ,df['clean_purpose'].str.contains('house tx district',na=False)
    ,df['clean_vendor'].str.contains('paychex', na=False)
    ,df['clean_vendor'].str.contains('cardmember services', na=False)
    ,df['clean_vendor'].str.contains('ascensus trust', na=False) 
    ,df['clean_purpose'].str.contains('text mess',na=False)  
    ,df['clean_purpose'].str.contains('donor',na=False)  
     ,df['clean_purpose'].str.contains('sms',na=False)
     ,df['clean_purpose'].str.contains('event',na=False)
     ,df['clean_purpose'].str.contains('meeting',na=False) 
     ,df['clean_purpose'].str.contains('polling',na=False) 
     ,df['clean_purpose'].str.contains('void',na=False)
    ,df['clean_purpose'].str.contains('credit card payment',na=False) 
    ,df['clean_purpose'].str.contains('accounting',na=False)  
    ,df['clean_purpose'].str.contains('transfer',na=False)
      ,df['clean_purpose'].str.contains('non federal',na=False) 
   ,df['clean_purpose'].str.contains('texting',na=False)
     ,df['clean_purpose'].str.contains('digital organizing',na=False)  
  ,df['clean_purpose'].str.contains('airfare',na=False)
    ,df['clean_purpose'].str.contains('digital prospecting',na=False) 
    ,df['clean_purpose'].str.contains('acqui',na=False)  
    ,df['clean_purpose'].str.contains('internet',na=False)  
    ,df['clean_purpose'].str.contains('storage',na=False)  
    ,df['clean_purpose'].str.contains('bank',na=False)  
    ,df['clean_purpose'].str.contains('retirement',na=False)  
    ,df['clean_purpose'].str.contains('email ',na=False)  
    ,df['clean_purpose'].str.contains('mail',na=False)  
    ,df['clean_purpose'].str.contains('supporter',na=False)  
    ,df['clean_purpose'].str.contains('online ad',na=False)  
    ,df['clean_purpose'].str.contains('digital ad',na=False)  
    ,df['clean_purpose'].str.contains('state pac',na=False)  
    ,df['clean_purpose'].str.contains('state party',na=False)  
    ,df['clean_purpose'].str.contains('professional development',na=False) 
    ,df['clean_purpose'].str.contains('pac cc transaction fees',na=False)  
    ,df['clean_vendor'].str.contains('visa',na=False)  
    ,df['clean_vendor'].str.contains('t mobile',na=False)  
    ,df['clean_vendor'].str.contains('chase',na=False)  
    ,df['clean_vendor'].str.contains('donor',na=False)  
    ,df['clean_vendor'].str.contains('truist',na=False) 
    ,df['clean_vendor'].str.contains('gusto',na=False)
    ,df['clean_vendor'].str.contains('hsp direct llc',na=False)
    ,df['clean_vendor'].str.contains('authorize net',na=False)  
    ,df['clean_vendor'].str.contains('amazon',na=False)  
    ,df['clean_vendor'].str.contains('facebook',na=False) 
    ,df['clean_purpose'].str.contains('gas',na=False)  
    ,df['clean_purpose'].str.contains('publication dues',na=False)  
    ,df['clean_vendor'].str.contains('intuit',na=False)
    ,df['clean_vendor'].str.contains('mailchimp',na=False)
    ,df['clean_vendor'].str.contains('comcast',na=False)
    ,df['clean_vendor'].str.contains('wix',na=False)
    ,df['clean_purpose'].str.contains('media',na=False)  
    ,df['clean_purpose'].str.contains('organiz',na=False)  
    ,df['clean_purpose'].str.contains(' ie',na=False) 
    ,df['clean_purpose'].str.contains('placement',na=False) 
    ,df['clean_purpose'].str.contains('buy',na=False) 
    ,df['clean_purpose'].str.contains('independent exp',na=False) 
    ,df['clean_purpose'].str.contains('bitcoin',na=False) 
    ,df['clean_purpose'].str.contains('ether',na=False) 
    ,df['clean_purpose'].str.contains('merchandise',na=False) 
    ,df['clean_purpose'].str.contains('list ',na=False)
    ,df['clean_purpose'].str.contains('staging lighting equipment',na=False)
    ,df['clean_purpose'].str.contains('message phone calls',na=False) 
    ,df['clean_purpose'].str.contains('usdc',na=False) 
    ,df['clean_purpose'].str.contains('tv ad',na=False) 
    ,df['clean_purpose'].str.contains('satellite ads',na=False)
    ,df['clean_purpose'].str.contains('artwork',na=False)
    ,df['clean_purpose'].str.contains('contrib',na=False)
    ,df['clean_purpose'].str.contains('to affil',na=False)
    ,df['clean_purpose'].str.contains('production',na=False)
    ,df['clean_purpose'].str.contains('compensation',na=False)
    ,df['clean_purpose'].str.contains('labor',na=False)
    ,df['clean_purpose'].str.contains('modeling',na=False)
    ,df['clean_purpose'].str.contains('membership political mobilization',na=False)
    ,df['clean_purpose'].str.contains('non fed political org natl',na=False)
    ,df['clean_purpose'].str.contains('association',na=False) 
    ,df['clean_purpose'].str.contains('state ind exp cmte',na=False) 
    ,df['clean_purpose'].str.contains('voter mobilization',na=False)
    ,df['clean_purpose'].str.contains('ie not yet disseminated',na=False)
    ,df['clean_purpose'].str.contains('phone calls',na=False)
    ,df['clean_purpose'].str.contains('overhead',na=False)
    ,df['clean_purpose'].str.contains('t shirt',na=False) 
    
    
    

]




values = ['crypto currency'
 , 'fundraising'
 ,'fundraising'
 ,'fundraising'
 ,'fundraising'
 ,'fundraising'
 ,'fundraising'
 ,'travel'
 ,'travel'
 ,'advertisements'
 ,'earmark'
 ,'advertisements'
 ,'salary and benefits'
 ,'salary and benefits'
 ,'salary and benefits'
 ,'salary and benefits'
 ,'travel'
 ,'travel'
 ,'contribution'
 ,'fundraising'
 ,'fundraising'
 ,'travel'
 ,'bank fees'
 ,'credit card payment'
 ,'advertisements'
 ,'fundraising'
 ,'consulting'
 ,'consulting'
 ,'fundraising'
 ,'fundraising'
 ,'technology'
 ,'overhead'
 ,'fundraising'
 ,'fundraising'
 ,'fundraising'
 ,'contribution'
 ,'contribution'
 ,'refund'
 ,'travel'
 ,'technology'
 ,'fundraising'
 ,'salary and benefits'
 ,'salary and benefits'
 ,'technology'
 ,'salary and benefits'
 ,'bank fees'
 ,'travel'
 ,'travel'
 ,'fundraising'
 ,'fundraising'
 ,'overhead'
 ,'travel'
 ,'overhead'
 ,'technology'
 ,'advertisements'
 ,'fundraising'
 ,'fundraising'
 ,'overhead'
 ,'fundraising'
 ,'fundraising'
 ,'fundraising'
 ,'fundraising'
 ,'sms, phonebanking, digital field'
 ,'sms, phonebanking, digital field'
 ,'fundraising'
 ,'sms, phonebanking, digital field'
 ,'fundraising'
 ,'fundraising'
 ,'consulting'
 ,'consulting'
 ,'travel'
 ,'fundraising'
 ,'fundraising'
 ,'salary and benefits'
 ,'fundraising'
 ,'contribution'
 ,'salary and benefits'
 ,'salary and benefits'
 ,'salary and benefits'
 ,'salary and benefits'
 ,'salary and benefits'
 ,'technology'
 ,'technology'
 ,'technology'
 ,'bank fees'
 ,'salary and benefits'
 ,'consulting'
 ,'travel'
 ,'consulting'
 ,'fundraising'
 ,'fundraising'
 ,'fundraising'
 ,'fundraising'
,'travel'
,'travel'
,'technology'
,'travel'
,'contribution'
,'travel'
,'contribution'
,'taxes'
,'taxes'
,'bank fees'
,'sms, phonebanking, digital field'
,'sms, phonebanking, digital field'
,'sms, phonebanking, digital field'
,'consulting'
,'overhead'
,'advertisements'
,'fundraising'
,'advertisements'
,'fundraising'
,'fundraising'
,'fundraising'
,'advertisements'
,'overhead'
,'overhead'
,'overhead'
,'overhead'
,'technology'
,'overhead'
,'overhead'
,'fundraising'
,'travel'
,'salary and benefits'
,'contribution'
,'salary and benefits'
,'credit card payment'
,'salary and benefits'
,'sms, phonebanking, digital field'
,'fundraising'
,'sms, phonebanking, digital field'
,'events'
,'events'
,'polling'
,'refund'
,'credit card payment'
,'overhead'
,'contribution'
,'contribution'
,'sms, phonebanking, digital field'
,'sms, phonebanking, digital field'
,'travel'
,'fundraising'
,'fundraising'
,'overhead'
,'overhead'
,'bank fees'
,'salary and benefits'
,'technology'
,'fundraising'
,'fundraising'
,'fundraising'
,'fundraising'
,'contribution'
,'contribution'
,'salary and benefits'
,'fundraising'
,'credit card payment'
,'technology'
,'credit card payment'
,'fundraising'
,'bank fees'
,'salary and benefits'
,'fundraising'
,'fundraising'
,'overhead'
,'fundraising'
,'travel'
,'overhead'
,'taxes'
,'technology'
,'overhead'
,'technology'
,'advertisements'
,'sms, phonebanking, digital field'
,'advertisements'
,'advertisements'
,'advertisements'
,'advertisements'
,'crypto currency'
,'crypto currency'
,'fundraising'
,'fundraising'
,'events'
,'sms, phonebanking, digital field'
,'crypto currency'
,'advertisements'
,'advertisements'
,'advertisements'
,'contribution'
,'contribution'
,'advertisements'
 ,'salary and benefits'
 ,'salary and benefits'
 ,'consulting'
 ,'sms, phonebanking, digital field'
 ,'contribution'
 ,'contribution'
 ,'contribution'
 ,'sms, phonebanking, digital field'
 ,'advertisements'
 ,'sms, phonebanking, digital field'
 ,'overhead'
 ,'fundraising'
]

# Apply conditions to create new column
df['exp_cat'] = np.select(conditions, values, default=None)

     

In [158]:
# What percent of the amount spent remains unclassified?
df[df['exp_cat'].isna()]['expenditure_amount'].sum()/df['expenditure_amount'].sum()

0.012978843705314286

In [159]:
# What percent of the lineitems remains unclassified?
df['exp_cat'].isna().sum()/df.shape[0]

0.03426573181173762

In [ ]:
# Fill all remaining unclassified categories as other
df['exp_cat']=df['exp_cat'].fillna('other')

In [161]:
# Number of lineitems in each category
df.groupby('exp_cat').size().sort_values(ascending=False)

exp_cat
fundraising                         166729
contribution                         82916
travel                               79317
salary and benefits                  55021
consulting                           50877
bank fees                            32344
technology                           29809
overhead                             23567
other                                19529
advertisements                        9722
credit card payment                   5727
earmark                               3113
sms, phonebanking, digital field      3093
events                                3084
refund                                2814
taxes                                 1402
polling                                675
crypto currency                        189
dtype: int64

In [162]:
# Amount spent in each category
df.groupby('exp_cat')['expenditure_amount'].sum().sort_values(ascending=False)

exp_cat
fundraising                         9.071709e+08
contribution                        6.682911e+08
consulting                          4.884865e+08
salary and benefits                 3.831003e+08
advertisements                      2.546937e+08
travel                              1.550771e+08
sms, phonebanking, digital field    7.188688e+07
crypto currency                     5.489457e+07
credit card payment                 5.222038e+07
overhead                            5.206980e+07
technology                          4.392728e+07
other                               4.216492e+07
polling                             3.125395e+07
bank fees                           1.655846e+07
events                              1.334941e+07
taxes                               1.163749e+07
refund                              1.730525e+06
earmark                             2.292377e+05
Name: expenditure_amount, dtype: float64

In [156]:
# Investigate remaining unclassified data
#df[df['exp_cat'].isna()].groupby(['clean_purpose']).size().sort_values(ascending=False).reset_index().head(50)
df[(df['exp_cat']=='other')].groupby('clean_purpose')['expenditure_amount'].sum().sort_values(ascending=False).reset_index().head(50)

,clean_purpose,expenditure_amount
0,disgorgement,7006618.33
1,,5839849.61
2,service fee,1169179.54
3,management fees,1140392.85
4,survey,677257.08
5,membership dues,522412.91
6,logistical and communications support,509000.00
7,message development and testing not distributed,440000.00
8,merchant fees,410224.56
9,in kind air columbia collier mgt th naples fl,378547.33


In [164]:
#Create a dataframe aggregating total spent in each category by cmte id 

by_cmte_exp_cat = pd.DataFrame(df.groupby(['filer_committee_id_number','exp_cat'])['expenditure_amount'].sum().reset_index())
by_cmte_exp_cat_wide = by_cmte_exp_cat.pivot(index='filer_committee_id_number', columns='exp_cat', values='expenditure_amount')

by_cmte_exp_cat_wide

exp_cat,advertisements,bank fees,consulting,contribution,credit card payment,crypto currency,earmark,events,fundraising,other,overhead,polling,refund,salary and benefits,"sms, phonebanking, digital field",taxes,technology,travel
filer_committee_id_number,,,,,,,,,,,,,,,,,,
C00000059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C00000422,NaN,28308.39,NaN,NaN,NaN,NaN,NaN,NaN,1069.14,148637.85,NaN,NaN,NaN,NaN,NaN,5518.00,NaN,NaN
C00000489,200.0,NaN,NaN,250.00,NaN,NaN,NaN,NaN,NaN,800.00,5544.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C00000729,NaN,13771.89,NaN,6400.00,NaN,NaN,NaN,NaN,32699.03,777.76,NaN,NaN,NaN,NaN,NaN,337.08,NaN,NaN
C00000885,NaN,300.00,12050.0,887550.00,NaN,NaN,NaN,NaN,NaN,46875.00,NaN,NaN,-16500.0,968.0,NaN,18137.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C00893826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12825.0,711.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C00893982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,671.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C00894022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# export agg df to csv
by_cmte_exp_cat_wide.reset_index().to_csv('../data/processed/cmte_exp_cat_agg.csv')

In [ ]:
# export processed df to csv 
df.to_csv('../data/processed/sb_opp_exp_cat.csv')